#  **1. 데이터 불러오기**

* 다운로드 받은 데이터 파일의 경로를 지정합니다.

In [ ]:
import os

TITANIC_PATH = "../input/titanic"

* csv파일을 판다스 데이터프레임으로 가져오는 함수를 만듭니다.

In [ ]:
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):


    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

* 테스트세트와 훈련세트를 데이터프레임으로 가져옵니다.

In [ ]:
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

* 훈련세트 확인

In [ ]:
train_data

* 테스트 세트 확인

 - 테스트 세트에는 레이블이 존재 하지 않습니다. 이것은 홈페이지에서 우리가 확인하기 위함이 목표이기 때문입니다.

In [ ]:
test_data

속성은 다음과 같은 의미를 가집니다:
* **Survived**: 타깃입니다. 0은 생존하지 못한 것이고 1은 생존을 의미합니다.
* **Pclass**: 승객 등급. 1, 2, 3등석.
* **Name**, **Sex**, **Age**: 이름 그대로 의미입니다.
* **SibSp**: 함께 탑승한 형제, 배우자의 수.
* **Parch**: 함께 탑승한 자녀, 부모의 수.
* **Ticket**: 티켓 아이디
* **Fare**: 티켓 요금 (파운드)
* **Cabin**: 객실 번호
* **Embarked**: 승객이 탑승한 곳. C(Cherbourg), Q(Queenstown), S(Southampton)

* 훈련데이터의 누락데이터를 확인하겠습니다.

In [ ]:
train_data.info()

 **Age**, **Cabin**, **Embarked** 속성의 일부가 null입니다(891개의 non-null 보다 작습니다). 
 특히 **Cabin**은 77%가 null입니다. 
 일단 **Cabin**은 무시하고 나머지를 활용하겠습니다. **Age**는 19%가 null이므로 이를 어떻게 처리할지 결정해야 합니다. null을 중간 나이로 바꾸는 것이 괜찮아 보입니다.

**Name**과 **Ticket** 속성도 값을 가지고 있지만 머신러닝 모델이 사용할 수 있는 숫자로 변환하는 것이 조금 까다롭습니다. 그래서 지금은 이 두 속성을 무시하겠습니다.

* 다음은 통계치를 확인해보겠습니다.

In [ ]:
train_data.describe()

*  38%만 **Survived**입니다. :( 거의 40%에 가까우므로 정확도를 사용해 모델을 평가해도 괜찮을 것 같습니다.
* 평균 **Fare**는 32.20 파운드라 그렇게 비싸보이지는 않습니다(아마 요금을 많이 반환해 주었기 때문일 것입니다)
* 평균 **Age**는 30보다 작습니다.

* 레이블이 0과 1로 되어있는지 확인해봅시다.

In [ ]:
train_data["Survived"].value_counts()

* 범주형 특성들을 확인해 보겠습니다 

In [ ]:
train_data["Pclass"].value_counts()

In [ ]:
train_data["Sex"].value_counts()

In [ ]:
train_data["Embarked"].value_counts()

**Embarked** 특성은 승객이 탑승한 곳을 알려 줍니다: C=Cherbourg, Q=Queenstown, S=Southampton.

# **2. 특성 전처리를 위한 파이프라인**

* 각열을 다르게 처리하기 위해 파이프라인과 DataFrameSelector 사용자 정의 클래스를 사용하겠습니다.

## 2.1 특정열을 선택 클래스 DataFrameSelector

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

## 2.2 숫자 특성 처리를 위한 파이프 라인 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["Age", "SibSp", "Parch", "Fare"])),
        ("imputer", SimpleImputer(strategy="median")),
    ])

In [ ]:
num_pipeline.fit_transform(train_data)

숫자 특성에 대해 파이프라인 처리를 한 것입니다.

## 2.3 범주형 특성 처리 파이프 라인

* 문자열의 범주형 특성 처리를 위해 별도의 imputer 클래스가 필요합니다.( SimpleImputer로는 할 수 없기 때문입니다.)

In [ ]:
# stackoverflow.com/questions/25239958 에서 착안했습니다
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

비어있는 값을 가장 많이 나오는 값으로 채워준다고 할 수 있다.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

* 범주형 특성을 위한 파이프 라인

In [ ]:
cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["Pclass", "Sex", "Embarked"])),
        ("imputer", MostFrequentImputer()),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])

In [ ]:
cat_pipeline.fit_transform(train_data)

문자열의 범주형 특성에 대해 파이프라인 처리를 한 것입니다.

## 2.4 전처리 파이프 라인 완성

* 이제 수치형과 범주형 파이프라인을 연결하여 만듭니다.

In [ ]:
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

머신러닝 모델을 훈련시키기위한 데이터 전처리 파이프를 완성하였습니다.

In [ ]:
X_train = preprocess_pipeline.fit_transform(train_data)
X_train

레이블도 가지고 옵니다.

In [ ]:
y_train = train_data["Survived"]

# **3.SVC 모델 훈련** 

## 3.1 SVC 분류기

SVC 모델에 훈련을 시킵니다.

In [ ]:
from sklearn.svm import SVC

svm_clf = SVC(gamma="auto")
svm_clf.fit(X_train, y_train)

이제 테스트 셋트에 대한 예측을 만들어서 홈페이지에서 검사를 맡을 수 있습니다.

In [ ]:
X_test = preprocess_pipeline.transform(test_data)
y_pred = svm_clf.predict(X_test)

하지만 좋은 점수를 위해 자체적으로 평가해보겠습니다.

## **3.2 모델 평가**

 교차검증

In [ ]:
from sklearn.model_selection import cross_val_score

svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
svm_scores.mean()

73% 정도입니다. 좀 더 높은 모델을 훈련시켜 보도록 하겠습니다.

RandomForestClassifier을 훈련시키고 교차검증 해보았습니다.

#  **4. RandomForestClassifier 모델**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

81% 로 성능이 상승 되었음을 알 수 있습니다.

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

plt.figure(figsize=(8, 4))
plt.plot([1]*10, svm_scores, ".")
plt.plot([2]*10, forest_scores, ".")
plt.boxplot([svm_scores, forest_scores], labels=("SVM","Random Forest"))
plt.ylabel("Accuracy", fontsize=14)
plt.show()

10 폴드 교차 검증에 대한 평균 정확도 대신 대신 모델에서 얻은 10개의 점수를 1사분위, 3사분위를 표현해주는 상자 수염 그림 그래프로 보면 **SVM에서 보다 Random Forest 모델**이 훨씬 더 1사분위에서 3사분위에 **박스 안에 모여 있고, 이상치(수염 밖의 값들은 표시 되지 않는데, 이것이 이상치이다.)도 훨씬 적음**을 알 수 있다. 즉, 더욱 성능이 높은 모델은 Random Forest 라고 할 수 있다. 

# **5. 성능 향상**

## **1. 동행자의 수**

부모, 자녀, 형제의 수로 하지말고 동행자의 수로 보는 것은 어떨지 특성을 변화시켜보자.

### 동행자 특성 추가

In [ ]:
train_data["RelativesOnboard"] = train_data["SibSp"] + train_data["Parch"]
train_data[["RelativesOnboard", "Survived"]].groupby(['RelativesOnboard']).mean()

부모,자녀와 형제를 묶어서 동행자로 나누어보았다. 그리고 생존률도 확인해보았다.

* 밑에서 훈련세트를 확인해보니 RelativesOnboard 열이 추가 되었음이 확인된다.

In [ ]:
train_data

In [ ]:
train_data[train_data["Survived"] == 1]["RelativesOnboard"].value_counts()


혼자만 탑승한 탑승객의 생존수가 가장 많다. 따라서 위의 전처리는 의미가 있을 것 같다.

### 파이프 라인 수정

* 수치형 파이프 라인을 수정한다. 부모&자녀, 형제 특성을 선택하지 않고 동행자수 특성을 선택하여 파이프 라인을 수정해주었다.

In [ ]:
num_pipeline1 = Pipeline([
        ("select_numeric", DataFrameSelector(["Age", "Fare", "RelativesOnboard"])),
        ("imputer", SimpleImputer(strategy="median")),
    ])

* 따라서 전체 전처리 파이프라인도 수정해주었다.

In [ ]:

preprocess_pipeline1 = FeatureUnion(transformer_list=[
        ("num_pipeline1", num_pipeline1),
        ("cat_pipeline", cat_pipeline),
    ])

* 파이프 라인에 넣어서 데이터를 전처리 해주었다.

In [ ]:
X_train1 = preprocess_pipeline1.fit_transform(train_data)
X_train1

### 훈련 및 평가

* 랜덤포레스트 모델을 훈련시키고 교차 검증해 보았다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_scores = cross_val_score(forest_clf, X_train1, y_train, cv=10)
forest_scores.mean()

80%로 오히려 성능이 더 안좋아졌다. 

아마도 전체 생존자 수는 혼자 다닌 사람이 많았지만, 혼자 다닌 사람의 생존률은 30%로 그다지 높지 않았다. 그래서 오히려 생존률과 동행자의 수가 밀접함이 확연하게 크지 않았던 것 같다.

## **2. 나이 범주화** 

그렇다면 여기다가 구체적인 나이보다 나이 범위 특성으로 설정하여 하는 것을 추가하여 보자.

### 나이 범주화 특성 추가

In [ ]:
train_data["AgeBucket"] = train_data["Age"] // 15 * 15
train_data[["AgeBucket", "Survived"]].groupby(['AgeBucket']).mean()

나이를 15로 나누어서 몪을 구하고 15를 곱해 15단위로 범주화 시켰다.

그리고 생존률도 확인해 보았다.

* 밑을 확인하면 훈련세트에 나이를 범주화 시켜 추가한 것이 확인된다.

In [ ]:
train_data

In [ ]:
train_data[train_data["Survived"] == 1]["AgeBucket"].value_counts()

나이에 따른 생존자 수를 보니 15세로 묶인 범주화가 가장 높고 다음 순으로 되어있다. 비교해보니 위의 생존률의 크기와 큰 연관이 없어 보이긴 하지만 일단 진행하겠다.

### 파이프 라인 수정

수치형 파이프라인에서 Age 대신 방금 추가한 AgeBucket 특성을 선택하게 해주어야한다.

In [ ]:
num_pipeline2 = Pipeline([
        ("select_numeric", DataFrameSelector(["AgeBucket", "Fare", "RelativesOnboard"])),
        ("imputer", SimpleImputer(strategy="median")),
    ])

따라서 전체 전처리 파이프 라인도 바뀐다. 

In [ ]:
preprocess_pipeline2 = FeatureUnion(transformer_list=[
        ("num_pipeline2", num_pipeline2),
        ("cat_pipeline", cat_pipeline),
    ])

* 파이프 라인에 넣어서 데이터를 전처리 해주었다.

In [ ]:
X_train2 = preprocess_pipeline2.fit_transform(train_data)
X_train2

### 훈련 및 평가

* 랜덤포레스트 모델을 훈련시키고 교차 검증해 보았다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_scores = cross_val_score(forest_clf, X_train2, y_train, cv=10)
forest_scores.mean()



81%로 성능이 향상 되었음을 볼 수 있다!!!

# 테스트 세트로 예측하기

테스트 세트에 앞에서 추가한 RelativesOnboard 특성과 AgeBucket 특성을 추가한다.

In [ ]:
test_data["RelativesOnboard"] = test_data["SibSp"] + test_data["Parch"]
test_data["AgeBucket"] = test_data["Age"] // 15 * 15

테스트 세트를 새로 바꾼 전처리 파이프로 넣어서 전처리를 해준다.

In [ ]:
X_test = preprocess_pipeline2.fit_transform(test_data)


* 원래 데이터로 모델을 훈련시키고 테스트 세트로 예측한다.

In [ ]:
forest_clf.fit(X_train2,  y_train)
submit = forest_clf.predict(X_test)
submit

제출을 csv 파일로 제출해야 하므로 파일을 바꿔준다


* numpy array → DataFrame로 바꿔주었다.

In [ ]:
submit_DataFrame = pd.DataFrame(submit)
submit_DataFrame

* 제출 양식에 맞추기 위해 새로운 데이터 프레임을 만들어 일단 PassengerId 특성을 넣어준다.

In [ ]:
submit_DataFrame1 = pd.DataFrame(test_data["PassengerId"])
submit_DataFrame1

* 테스트세트에 대한 예측값들을 넣어준다.

In [ ]:
submit_DataFrame1["Survived"] = submit_DataFrame[0]
submit_DataFrame1 

* 양식을 맞춰주기 위해 PassengerId를 인덱스로 바꾼다.

In [ ]:
submit_DataFrame1.set_index('PassengerId', inplace=True)
submit_DataFrame1

 * DataFrame → csv 파일로 바꿔주었다.

In [ ]:
submit_DataFrame1.to_csv('submit1.csv',sep=',', na_rep='NaN')



